# 04 - Filter

In [1]:
# imports
import pandas as pd
import pickle
import sqlite3

from sqlalchemy import create_engine

In [2]:
rec_df = pickle.load(open('../Data/filter_rec_df.pkl', 'rb'))

In [3]:
rec_df.head()

,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,...,legalities_vintage,legalities_pauper,legalities_future,legalities_commander,legalities_historic,legalities_brawl,legalities_modern,legalities_duel,cmc,card_link
name,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000,0.873,0.923,0.902,0.905,0.777,0.904,0.903,0.937,0.830,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,https://scryfall.com/card/7ed/319/static-orb?u...
Sensory Deprivation,0.873,0.000,0.922,0.740,0.734,0.743,0.879,0.704,0.786,0.768,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,https://scryfall.com/card/m14/71/sensory-depri...
Road of Return,0.923,0.922,0.000,0.955,0.884,0.887,0.882,0.907,0.883,0.865,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,https://scryfall.com/card/c19/34/road-of-retur...
Storm Crow,0.902,0.740,0.955,0.000,0.772,0.856,0.916,0.830,0.819,0.799,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,https://scryfall.com/card/fmb1/31/storm-crow?u...
Walking Sponge,0.905,0.734,0.884,0.772,0.000,0.856,0.866,0.477,0.797,0.646,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,https://scryfall.com/card/ulg/47/walking-spong...


In [4]:
# connect to SQL database
conn = sqlite3.connect('../Data/MTG_Recommender.db')

In [5]:
sql = """
SELECT * FROM filter
LIMIT 5
"""

list(pd.read_sql_query(sql, conn).columns)

['name',
 'colors_B',
 'color_identity_B',
 'colors_G',
 'color_identity_G',
 'colors_R',
 'color_identity_R',
 'colors_U',
 'color_identity_U',
 'colors_W',
 'color_identity_W',
 'card_type_Creature',
 'card_type_Instant',
 'card_type_Enchantment',
 'card_type_Sorcery',
 'card_type_Artifact',
 'card_type_Land',
 'card_type_Planeswalker',
 'card_type_Tribal',
 'sub_type_Plant',
 'sub_type_Arlinn',
 'sub_type_Bird',
 'sub_type_Kraken',
 'sub_type_Power-Plant',
 'sub_type_Ouphe',
 'sub_type_Praetor',
 'sub_type_Juggernaut',
 'sub_type_Rebel',
 'sub_type_Chandra',
 'sub_type_Ogre',
 'sub_type_Spike',
 'sub_type_Orgg',
 'sub_type_Slith',
 'sub_type_Rigger',
 'sub_type_Leviathan',
 'sub_type_Ox',
 'sub_type_Trilobite',
 'sub_type_Cockatrice',
 'sub_type_Pilot',
 'sub_type_Artificer',
 'sub_type_Homunculus',
 'sub_type_Arcane',
 'sub_type_Cat',
 'sub_type_Brushwagg',
 'sub_type_Nixilis',
 'sub_type_Specter',
 'sub_type_Elephant',
 'sub_type_Equipment',
 'sub_type_Rowan',
 'sub_type_Atog',
 '

In [6]:
# testing for flask app
# sample of what filters might be
u_sub_type = 'Human'
u_color = ['R']
u_color_id = ['R']
u_card_type = 'None'
u_cmc_lower = 0
u_cmc_upper = 4
u_legalities = 'modern'

# build the sql query piece by piece
sql = """
SELECT name FROM filter
WHERE 
"""
# card type filter
if u_card_type != 'None':
    sql += f"card_type_{u_card_type} != 0 "
else:
    sql += f"card_type_Creature < 2 " # need a place to start that doesn't begin with 'AND' if no cardtype is used

# subtype filter
if u_sub_type != 'None':
    sql += f"AND sub_type_{u_sub_type} != 0 "

# color filter
if u_color != []:
    for color in u_color:
        sql += f"AND colors_{color} != 0 "

# color id filter
if u_color_id != []:
    for colorid in u_color_id:
        sql += f"AND color_identity_{colorid} != 0 "
        
# cmc filter
if u_cmc_upper != 'None':
    sql += f"AND cmc BETWEEN {u_cmc_lower} AND {u_cmc_upper} "

# format filter
if u_legalities != 'None':
    sql += f"AND legalities_{u_legalities} != 0"


filtered_list = list(pd.read_sql_query(sql, conn)['name'].values)
filtered_list

['Keldon Raider',
 'Soul-Scar Mage',
 'Planebound Accomplice',
 'Akroan Line Breaker',
 'Reckless Cohort',
 'Embraal Gear-Smasher',
 'Seasoned Pyromancer',
 'Winota, Joiner of Forces',
 'Blood Cultist',
 'Nef-Crop Entangler',
 'Kolaghan Stormsinger',
 'Kolaghan Forerunners',
 'League Guildmage',
 'Abbot of Keral Keep',
 'Stormfist Crusader',
 'Lyzolda, the Blood Witch',
 'Vulshok Heartstoker',
 'Paragon of Fierce Defiance',
 'Valduk, Keeper of the Flame',
 'Captivating Crew',
 'Forerunner of the Empire',
 'Borderland Marauder',
 'Sandstone Warrior',
 'Brazen Freebooter',
 'Fresh-Faced Recruit',
 'Humble Defector',
 'Harsh Mentor',
 'Storm Herald',
 'Stormcaller of Keranos',
 'Valakut Invoker',
 'Gnat Alley Creeper',
 'Reckless Fireweaver',
 'Hero of the Nyxborn',
 'Keldon Overseer',
 'Leaping Master',
 'Vithian Stinger',
 'Fervent Paincaster',
 'Whip Sergeant',
 'Rakdos Augermage',
 'Burning Prophet',
 'Lightning Berserker',
 'Daring Buccaneer',
 'Fallen Ferromancer',
 'Thermo-Alchemis

In [7]:
rec_df[filtered_list].T['Shock'].sort_values()[0:10]

Shock Troops                                0.247
Skirsdag Cultist                            0.312
Grim Lavamancer                             0.365
Mardu Heart-Piercer                         0.393
Valakut Invoker                             0.414
Fearless Halberdier                         0.428
Labyrinth Champion                          0.443
Sell-Sword Brute                            0.445
Vulshok Sorcerer                            0.453
Embereth Shieldbreaker // Battle Display    0.458
Name: Shock, dtype: float64